In [1]:
import cv2
import numpy as np


def image_preprocessing(image):
    def check_image_size(img):
        if img.shape[0] < 224 or img.shape[1] < 224:
            return False
        else:
            return True
    def check_retina_exist(img):
        center_x, center_y = image.shape[1] // 2, image.shape[0] // 2
        radius = min(center_x, center_y) - 10  # Add a margin of 10 pixels
        mask = np.zeros(image.shape[:2], np.uint8)
        cv2.circle(mask, (center_x, center_y), radius, 255, -1)
        masked_image = cv2.bitwise_and(image, image, mask=mask)
        if cv2.countNonZero(masked_image) < 1000:
            return False
        else:
            return True
    def crop_image(img, tol=7):
        if img.ndim == 2:
            mask = img > tol
            return img[np.ix_(mask.any(1), mask.any(0))]
        elif img.ndim == 3:
            gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            mask = gray_img > tol
            check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
            if check_shape == 0:  # image is too dark so that we crop out everything,
                return img  # return original image
            else:
                img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
                img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
                img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
                img = np.stack([img1, img2, img3], axis=-1)

            return img

    def circle_crop(img):
        img = crop_image(img)

        height, width, depth = img.shape
        largest_side = np.max((height, width))
        img = cv2.resize(img, (largest_side, largest_side))

        height, width, depth = img.shape

        x = width // 2
        y = height // 2
        r = np.amin((x, y))

        circle_img = np.zeros((height, width), np.uint8)
        cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
        img = cv2.bitwise_and(img, img, mask=circle_img)
        img = crop_image(img)

        return img
    
    check_size=check_image_size(image)
    check_retina=check_retina_exist(image)
    if (check_size==False):
        return "Image dimensions are too small for diabetic retinopathy classification."
    elif (check_retina==False):
        return "Image does not contain the retina."
    else:
        image = crop_image(image)
        image = circle_crop(image)
        gaussian = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), 10), -4, 128)
        gaussian = cv2.resize(gaussian, (224, 224))
        return gaussian
